In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [9]:
torch.manual_seed(42)

In [10]:
df = pd.read_csv('../loan_approval_dataset.csv')

In [11]:
df.head()

loan_id   no_of_dependents      education  self_employed   income_annum  \
0        1                  2       Graduate             No        9600000   
1        2                  0   Not Graduate            Yes        4100000   
2        3                  3       Graduate             No        9100000   
3        4                  3       Graduate             No        8200000   
4        5                  5   Not Graduate            Yes        9800000   

    loan_amount   loan_term   cibil_score   residential_assets_value  \
0      29900000          12           778                    2400000   
1      12200000           8           417                    2700000   
2      29700000          20           506                    7100000   
3      30700000           8           467                   18200000   
4      24200000          20           382                   12400000   

    commercial_assets_value   luxury_assets_value   bank_asset_value  \
0                  17600000              22700000            8000000   
1                   2200000               8800000            3300000   
2                   4500000              33300000           12800000   
3                   3300000              23300000            7900000   
4                   8200000              29400000            5000000   

   loan_status  
0     Approved  
1     Rejected  
2     Rejected  
3     Rejected  
4     Rejected

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   loan_id                    4269 non-null   int64 
 1    no_of_dependents          4269 non-null   int64 
 2    education                 4269 non-null   object
 3    self_employed             4269 non-null   object
 4    income_annum              4269 non-null   int64 
 5    loan_amount               4269 non-null   int64 
 6    loan_term                 4269 non-null   int64 
 7    cibil_score               4269 non-null   int64 
 8    residential_assets_value  4269 non-null   int64 
 9    commercial_assets_value   4269 non-null   int64 
 10   luxury_assets_value       4269 non-null   int64 
 11   bank_asset_value          4269 non-null   int64 
 12   loan_status               4269 non-null   object
dtypes: int64(10), object(3)
memory usage: 433.7+ KB


In [13]:
df.describe()

loan_id   no_of_dependents   income_annum   loan_amount  \
count  4269.000000        4269.000000   4.269000e+03  4.269000e+03   
mean   2135.000000           2.498712   5.059124e+06  1.513345e+07   
std    1232.498479           1.695910   2.806840e+06  9.043363e+06   
min       1.000000           0.000000   2.000000e+05  3.000000e+05   
25%    1068.000000           1.000000   2.700000e+06  7.700000e+06   
50%    2135.000000           3.000000   5.100000e+06  1.450000e+07   
75%    3202.000000           4.000000   7.500000e+06  2.150000e+07   
max    4269.000000           5.000000   9.900000e+06  3.950000e+07   

         loan_term   cibil_score   residential_assets_value  \
count  4269.000000   4269.000000               4.269000e+03   
mean     10.900445    599.936051               7.472617e+06   
std       5.709187    172.430401               6.503637e+06   
min       2.000000    300.000000              -1.000000e+05   
25%       6.000000    453.000000               2.200000e+06   
50%      10.000000    600.000000               5.600000e+06   
75%      16.000000    748.000000               1.130000e+07   
max      20.000000    900.000000               2.910000e+07   

        commercial_assets_value   luxury_assets_value   bank_asset_value  
count              4.269000e+03          4.269000e+03       4.269000e+03  
mean               4.973155e+06          1.512631e+07       4.976692e+06  
std                4.388966e+06          9.103754e+06       3.250185e+06  
min                0.000000e+00          3.000000e+05       0.000000e+00  
25%                1.300000e+06          7.500000e+06       2.300000e+06  
50%                3.700000e+06          1.460000e+07       4.600000e+06  
75%                7.600000e+06          2.170000e+07       7.100000e+06  
max                1.940000e+07          3.920000e+07       1.470000e+07

In [14]:
df.columns = df.columns.str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.columns

C:\Users\user\AppData\Local\Temp\ipykernel_11060\728488614.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Index(['loan_id', 'no_of_dependents', 'education', 'self_employed',
       'income_annum', 'loan_amount', 'loan_term', 'cibil_score',
       'residential_assets_value', 'commercial_assets_value',
       'luxury_assets_value', 'bank_asset_value', 'loan_status'],
      dtype='object')

In [15]:
df['debt_to_income_ratio'] = df['loan_amount'] / df['income_annum']
df['total_assets_value'] = (df['residential_assets_value'] + df['commercial_assets_value'] +
                            df['luxury_assets_value'] + df['bank_asset_value'])
df['loan_to_asset_ratio'] = df['loan_amount'] / df['total_assets_value']

In [16]:
df.head()

loan_id  no_of_dependents     education self_employed  income_annum  \
0        1                 2      Graduate            No       9600000   
1        2                 0  Not Graduate           Yes       4100000   
2        3                 3      Graduate            No       9100000   
3        4                 3      Graduate            No       8200000   
4        5                 5  Not Graduate           Yes       9800000   

   loan_amount  loan_term  cibil_score  residential_assets_value  \
0     29900000         12          778                   2400000   
1     12200000          8          417                   2700000   
2     29700000         20          506                   7100000   
3     30700000          8          467                  18200000   
4     24200000         20          382                  12400000   

   commercial_assets_value  luxury_assets_value  bank_asset_value loan_status  \
0                 17600000             22700000           8000000    Approved   
1                  2200000              8800000           3300000    Rejected   
2                  4500000             33300000          12800000    Rejected   
3                  3300000             23300000           7900000    Rejected   
4                  8200000             29400000           5000000    Rejected   

   debt_to_income_ratio  total_assets_value  loan_to_asset_ratio  
0              3.114583            50700000             0.589744  
1              2.975610            17000000             0.717647  
2              3.263736            57700000             0.514731  
3              3.743902            52700000             0.582543  
4              2.469388            55000000             0.440000

In [17]:
print(df['education'].value_counts())
print("")
print(df['self_employed'].value_counts())
print("")
print(df['loan_status'].value_counts())

education
Graduate        2144
Not Graduate    2125
Name: count, dtype: int64

self_employed
Yes    2150
No     2119
Name: count, dtype: int64

loan_status
Approved    2656
Rejected    1613
Name: count, dtype: int64


In [18]:

df['education'] = df['education'].apply(lambda x: 1 if x == 'Graduate' else 0)
df['self_employed'] = df['self_employed'].apply(lambda x: 1 if x == 'Yes' else 0)
df['loan_status'] = df['loan_status'].apply(lambda x: 1 if x == 'Approved' else 0)
df.head()

loan_id  no_of_dependents  education  self_employed  income_annum  \
0        1                 2          1              0       9600000   
1        2                 0          0              1       4100000   
2        3                 3          1              0       9100000   
3        4                 3          1              0       8200000   
4        5                 5          0              1       9800000   

   loan_amount  loan_term  cibil_score  residential_assets_value  \
0     29900000         12          778                   2400000   
1     12200000          8          417                   2700000   
2     29700000         20          506                   7100000   
3     30700000          8          467                  18200000   
4     24200000         20          382                  12400000   

   commercial_assets_value  luxury_assets_value  bank_asset_value  \
0                 17600000             22700000           8000000   
1                  2200000              8800000           3300000   
2                  4500000             33300000          12800000   
3                  3300000             23300000           7900000   
4                  8200000             29400000           5000000   

   loan_status  debt_to_income_ratio  total_assets_value  loan_to_asset_ratio  
0            1              3.114583            50700000             0.589744  
1            0              2.975610            17000000             0.717647  
2            0              3.263736            57700000             0.514731  
3            0              3.743902            52700000             0.582543  
4            0              2.469388            55000000             0.440000

In [19]:
# Define the CIBIL rating function
def cibil_rating(value):
    if 300 <= value <= 549:
        return "Poor"
    elif 550 <= value <= 649:
        return "Average"
    elif 650 <= value <= 749:
        return "Good"
    elif 750 <= value <= 900:
        return "Excellent"
    else:
        return "Error"

# Create a new column 'CIBIL_rating' using the cibil_rating function
df['CIBIL_rating'] = df['cibil_score'].apply(cibil_rating)

# Grouping data based on the CIBIL Rating and Loan approval Status
cibil_rating_df = df.groupby(['CIBIL_rating', 'loan_status']).size().reset_index(name='Total')

In [20]:
cibil_rating_df

CIBIL_rating  loan_status  Total
0      Average            0      2
1      Average            1    681
2    Excellent            0      6
3    Excellent            1   1050
4         Good            0      5
5         Good            1    740
6         Poor            0   1600
7         Poor            1    185

In [21]:
plt.figure(figsize=(10, 6))
sns.barplot(x='CIBIL_rating', y='Total', hue='loan_status', data=cibil_rating_df, palette='coolwarm')

# Creating Labels
plt.title("Loan Approval Status by CIBIL Rating")
plt.xlabel("CIBIL Rating")
plt.ylabel("Count")

# Show the plot
plt.show()

In [22]:
df.drop(columns=['CIBIL_rating'], inplace=True)

In [23]:
corr_matrix = df.corr()
corr_matrix
plt.figure(figsize=(10, 8))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, square=True)

# Set the title and labels
plt.title('Correlation Matrix')
plt.show()

In [24]:
df

loan_id  no_of_dependents  education  self_employed  income_annum  \
0           1                 2          1              0       9600000   
1           2                 0          0              1       4100000   
2           3                 3          1              0       9100000   
3           4                 3          1              0       8200000   
4           5                 5          0              1       9800000   
...       ...               ...        ...            ...           ...   
4264     4265                 5          1              1       1000000   
4265     4266                 0          0              1       3300000   
4266     4267                 2          0              0       6500000   
4267     4268                 1          0              0       4100000   
4268     4269                 1          1              0       9200000   

      loan_amount  loan_term  cibil_score  residential_assets_value  \
0        29900000         12          778                   2400000   
1        12200000          8          417                   2700000   
2        29700000         20          506                   7100000   
3        30700000          8          467                  18200000   
4        24200000         20          382                  12400000   
...           ...        ...          ...                       ...   
4264      2300000         12          317                   2800000   
4265     11300000         20          559                   4200000   
4266     23900000         18          457                   1200000   
4267     12800000          8          780                   8200000   
4268     29700000         10          607                  17800000   

      commercial_assets_value  luxury_assets_value  bank_asset_value  \
0                    17600000             22700000           8000000   
1                     2200000              8800000           3300000   
2                     4500000             33300000          12800000   
3                     3300000             23300000           7900000   
4                     8200000             29400000           5000000   
...                       ...                  ...               ...   
4264                   500000              3300000            800000   
4265                  2900000             11000000           1900000   
4266                 12400000             18100000           7300000   
4267                   700000             14100000           5800000   
4268                 11800000             35700000          12000000   

      loan_status  debt_to_income_ratio  total_assets_value  \
0               1              3.114583            50700000   
1               0              2.975610            17000000   
2               0              3.263736            57700000   
3               0              3.743902            52700000   
4               0              2.469388            55000000   
...           ...                   ...                 ...   
4264            0              2.300000             7400000   
4265            1              3.424242            20000000   
4266            0              3.676923            39000000   
4267            1              3.121951            28800000   
4268            1              3.228261            77300000   

      loan_to_asset_ratio  
0                0.589744  
1                0.717647  
2                0.514731  
3                0.582543  
4                0.440000  
...                   ...  
4264             0.310811  
4265             0.565000  
4266             0.612821  
4267             0.444444  
4268             0.384217  

[4269 rows x 16 columns]

In [25]:
df_clean = df.to_csv("../loan_approval_dataset_cleaned")